<a href="https://colab.research.google.com/github/sarang-manohar/Coursera_Capstone/blob/main/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook puts in the seed of the Capstone Project with Coursera!

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import folium
from pandas.io.json import json_normalize
print('Libraries imported')

Libraries imported


In [2]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(html_data.text, 'html.parser')
table = soup.find("table",{"rules":"all"})

table_data = []

for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_data.append(cell)

df=pd.DataFrame(table_data)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df['Address'] = df['Borough']+', Toronto, Ontario, Canada, '+df['PostalCode']
df.head()

,PostalCode,Borough,Neighborhood,Address
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A"
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A"
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A"


In [3]:
lat=[]
long=[]
for i, pc in enumerate(df['PostalCode']):
    with requests.Session() as session:
        g = geocoder.arcgis('{}, Toronto, Canada'.format(pc),session=session)
    lat.append(g.latlng[0])
    long.append(g.latlng[1])

df['Latitude'] = lat
df['Longitude'] = long

df.head()

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A",43.66253,-79.39188
